Initial set up, including search terms.

In [ ]:
%run "Common setup.ipynb"

In [ ]:
import restful_wos

In [ ]:
# Create client and send query
client = restful_wos.RESTClient(WOS_CONFIG)
search_request = 'TS=("sensitivity analysis" OR "uncertainty analysis" OR "uncertainty quantification" OR "uncertainty propagation" OR "local sensitivity analysis" OR "LSA" OR "one-at-a-time" OR "exploratory modeling" OR "OAT" OR "global sensitivity analysis" OR "GSA" OR "all-at-a-time" OR "AAT") AND WC=("ENVIRONMENTAL SCIENCES" OR "WATER RESOURCES" OR "ENGINEERING ENVIRONMENTAL" OR "INTERDISCIPLINARY APPLICATIONS")'
resp = client.query(search_request, time_span=('2000-01-01', '2017-12-31'))

# Convert parsed responses into RIS records
ris_data = restful_wos.to_ris_text(resp)

# Output to a txt file
requested_corpora_fn = "{}/rest_requested_corpora.txt".format(TMP_DIR)
restful_wos.write_file(ris_data, requested_corpora, overwrite=True)

In [ ]:
RC = mk.RecordCollection(requested_corpora_fn)

In [ ]:
print("Corpora consists of", len(RC), "Publications")

In [ ]:
from datetime import datetime
repset_df = pd.DataFrame(RC.forNLP(extraColumns=["AU", "SO", "DE"]))
repset_df = repset_df.drop(["keywords", "abstract"], axis=1)

assert len(repset_df.id.unique()) == len(repset_df.id), "Duplicate records found!"

repset_fn = "{}/{}.csv".format(DATA_DIR, RC)
with open(repset_fn, 'w') as fn:
    fn.write("# Data from Clarivate Analytics' Web of Science, retrieved 5 December 2018\n")
    fn.write("# This file generated on {}\n".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    repset_df.index.name = "item"
    repset_df.to_csv(fn)